In [1]:
import spacy
from fuzzywuzzy import fuzz

# Lade das deutsche spaCy-Modell
nlp = spacy.load("de_core_news_md")

# Funktion zum Laden von Namen aus Datei
def load_names(path, label):
    with open(path, "r", encoding="utf-8") as f:
          names = [name.strip() for name in f if name.strip()]
          patterns = [{"label": label, "pattern": name} for name in names]
    return patterns, names

# Gazetteer laden
vornamen_patterns, vornamen_liste = load_names("Vornamen.txt", "VORNAME")
nachnamen_patterns, nachnamen_liste = load_names("Nachnamen.txt", "NACHNAME")
titel_patterns, titel_liste = load_names("Titel.txt", "TITEL")
wohnort_patterns, wohnort_liste = load_names("Orte.txt", "WOHNORT")
postleitzahl_patterns, postleitzahl_liste = load_names("Postleitzahlen.txt", "POSTLEITZAHL")
strasse_patterns = [
    {
        "label": "STRASSE",
        "pattern": [
            {"TEXT": {"REGEX": r".*(straße|gasse|allee|weg|platz|str.|grund)$"}},
            {"TEXT": {"REGEX": r"^\d+[a-zA-Z]?$"}}
        ]
    }
]
vertragsnummer_patterns = [
    {
        "label": "VERTRAGSNUMMER",
        "pattern": [
            {"LOWER": {"IN": ["vertragsnummer", "vertragsnr.", "vnr", "vn"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d{6,12}\.?$"}}

        ]
    }
]

kundennummer_patterns = [
    {
        "label": "KUNDENNUMMER",
        "pattern": [
            {"LOWER": {"IN": ["kundennummer", "kundennr.", "kdnr", "kd"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d{6,12}\.?$"}}
        ]
    }
]

zuordnungsnummer_patterns = [
    {
        "label": "ZUORDNUNGSNUMMER",
        "pattern": [
            {"LOWER": {"IN": ["znr", "zuordnungsnummer"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d{6,12}\.?$"}}
        ]
    }
]
iban_pattern = [
    {"label": "IBAN", "pattern": [{"TEXT": {"REGEX": r"^[A-Z]{2}[0-9]{2}[A-Z0-9]{11,30}$"}}]}
]

bic_pattern = [
    {"label": "BIC", "pattern": [{"TEXT": {"REGEX": r"^[A-Z]{6}[A-Z2-9][A-NP-Z0-9]([A-Z0-9]{3})?$"}}]}
]

zahlung_pattern = [
    {
        "label": "ZAHLUNG",
        "pattern": [
            {"TEXT": {"REGEX": r"^\d+[.,]?\d{0,2}$"}},
            {"TEXT": {"REGEX": r"^(€|euro|eur)$"}}
        ]
    }
]

zählerstand_patterns = [
    {
        "label": "ZÄHLERSTAND",
        "pattern": [
            {"LOWER": {"IN": ["zählerstand"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d+(\.\d+)?$"}}
        ]
    }
]

zählernummer_patterns = [
    {
        "label": "ZÄHLERNUMMER",
        "pattern": [
            {"LOWER": {"IN": ["zählernummer"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^[A-Z0-9]{6,20}$"}}  # Groß- und Kleinbuchstaben + Ziffern erlaubt
        ]
    }
]

verbrauch_patterns = [
    {
        "label": "VERBRAUCH",
        "pattern": [
            {"LOWER": {"IN": ["verbrauch"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d+(\.\d+)?$"}},
            {"LOWER": {"IN": ["kwh", "m³", "kw"]}, "OP": "?"}
        ]
    }
]

verbrauch_patterns += [
    {
        "label": "VERBRAUCH",
        "pattern": [
            {"LOWER": {"IN": ["verbrauch"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d+(?:[.,]\d+)?(kwh|m³|kw)$"}}
        ]
    }
]


wlv_patterns = [
    {
        "label": "WLV",
        "pattern": [
            {"LOWER": {"IN": ["wlv"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d{4,12}$"}}
        ]
    }
]

email_pattern = [
    {
        "label": "EMAIL",
        "pattern": [
            {"TEXT": {"REGEX": r"^[\w\.-]+@[\w\.-]+\.\w{2,}$"}}
        ]
    }
]

telefon_pattern = [
    {
        "label": "TELEFON",
        "pattern": [
            {"TEXT": {"REGEX": r"^(\+49|0)[\d\s/-]{7,}$"}}
        ]
    }
]

url_pattern = [
    {
        "label": "URL",
        "pattern": [
            {"TEXT": {"REGEX": r"^https?://[\w\-\.]+\.\w{2,}(/[\w\-\.]*)*$"}}
        ]
    },
    {
        "label": "URL",
        "pattern": [
            {"TEXT": {"REGEX": r"^www\.[\w\-\.]+\.\w{2,}(/[\w\-\.]*)*$"}}
        ]
    }
]

datum_pattern = [
    {
        "label": "DATUM",
        "pattern": [
            {"TEXT": {"REGEX": r"^(\d{1,2}[./-]){2}\d{2,4}$"}}  # z. B. 15.06.2024
        ]
    },
    {
        "label": "DATUM",
        "pattern": [
            {"TEXT": {"REGEX": r"^\d{4}-\d{2}-\d{2}$"}}  # z. B. 2024-06-15
        ]
    },
    {
        "label": "DATUM",
        "pattern": [
            {"TEXT": {"REGEX": r"^\d{1,2}$"}},  # z. B. 15
            {"LOWER": {"IN": [
                "januar", "jan", "februar", "feb", "märz", "maerz", "mrz", "april", "apr",
                "mai", "juni", "jun", "juli", "jul", "august", "aug", "september", "sep",
                "oktober", "okt", "november", "nov", "dezember", "dez"
            ]}},
            {"TEXT": {"REGEX": r"^\d{2,4}$"}, "OP": "?"}  # optional Jahr
        ]
    }
]





# EntityRuler erstellen und Muster hinzufügen
ruler = nlp.add_pipe("entity_ruler", before="ner")
ruler.add_patterns(vornamen_patterns + nachnamen_patterns + titel_patterns + postleitzahl_patterns + wohnort_patterns + strasse_patterns + vertragsnummer_patterns + kundennummer_patterns + zuordnungsnummer_patterns + iban_pattern + bic_pattern + zahlung_pattern + zählerstand_patterns + zählernummer_patterns + verbrauch_patterns + wlv_patterns+ email_pattern + telefon_pattern + url_pattern + datum_pattern)  # 👈 Muster hinzufügen!

# Beispieltext
text = "Hallo liebes Eon Team, es geht um die Vertragsnummer 406027919. Bei der Einrichtung meines neuen Vertrages wurde leider die Überweisung als Zahlungsart gewählt von dem jungen Kollegen an der Wohnungstür. Ich würde es gerne wieder per Lastschrift abbuchen lassen, um mir den Stress zu ersparen. Verbraucherstelle ist weiterhin die Gertzgasse 2 in 17389 Anklam. Gruß Berthold Huhn"

doc = nlp(text)

# Ausgabe der erkannten Entitäten
for ent in doc.ents:
    print(ent.text, ent.label_)

anonymized = doc.text
for ent in doc.ents:
    if ent.label_ in {"VORNAME", "NACHNAME", "TITEL", "POSTLEITZAHL", "WOHNORT", "STRASSE", "VERTRAGSNUMMER", "KUNDENNUMMER", "ZUORDNUNGSNUMMER", "IBAN", "BIC", "ZAHLUNG", "ZÄHLERSTAND", "ZÄHLERNUMMER", "VERBRAUCH", "WLV", "EMAIL", "TELEFON", "URL", "DATUM"}:
        anonymized = anonymized.replace(ent.text, f"[{ent.label_}]")

print("\nAnonymisiert:")
print(anonymized)

print("\n🔍 Fuzzy Matches (Ähnlichkeit > 90):")
for token in doc:
    name = token.text
    for referenz_name in vornamen_liste:
        if fuzz.ratio(name, referenz_name) > 90:
            print(f"Mögliches VORNAME-Fuzzy-Match: {name} ≈ {referenz_name}")
            break
    for referenz_name in nachnamen_liste:
        if fuzz.ratio(name, referenz_name) > 90:
            print(f"Mögliches NACHNAME-Fuzzy-Match: {name} ≈ {referenz_name}")
            break
    for referenz_name in titel_liste:
        if fuzz.ratio(name, referenz_name) > 99:
            print(f"Mögliches TITEL-Fuzzy-Match: {name} ≈ {referenz_name}")
            break



/opt/anaconda3/envs/piranha-env/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Hallo liebes Eon Team MISC
Vertragsnummer 406027919. VERTRAGSNUMMER
Verbraucherstelle LOC
Gertzgasse 2 STRASSE
17389 POSTLEITZAHL
Anklam WOHNORT
Berthold VORNAME

Anonymisiert:
Hallo liebes Eon Team, es geht um die [VERTRAGSNUMMER] Bei der Einrichtung meines neuen Vertrages wurde leider die Überweisung als Zahlungsart gewählt von dem jungen Kollegen an der Wohnungstür. Ich würde es gerne wieder per Lastschrift abbuchen lassen, um mir den Stress zu ersparen. Verbraucherstelle ist weiterhin die [STRASSE] in [POSTLEITZAHL] [WOHNORT]. Gruß [VORNAME] Huhn

🔍 Fuzzy Matches (Ähnlichkeit > 90):
Mögliches VORNAME-Fuzzy-Match: Berthold ≈ Berthold
